L'objectif de ce notebook est de procéder à une jointure entre les données du fichier "Films les plus diffusés à la télévision.xlsx" qui regroupe les films ayant été diffusé le plus à la TV avec un dataset de TMDB qui apporte davantage d'informations pour chaque film.


Vincent Lucas, 01/2025

In [ ]:
pip install Wikipedia-API

  Preparing metadata (setup.py) ... done
  Created wheel for Wikipedia-API: filename=Wikipedia_API-0.7.3-py3-none-any.whl size=14325 sha256=09b3b1b5638ea08746243b4f1ae399d67fe8c7fd03198b57e7f7ae66fb12d397
  Stored in directory: /root/.cache/pip/wheels/39/c8/cd/ba9a385ad2f6e457226fe2a7dceaeb0c17a19346d2e78ff4be
Successfully built Wikipedia-API


In [ ]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os, re
import wikipediaapi
from rapidfuzz import fuzz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chemin_donnees = "/content/drive/MyDrive/PIP2025/Donnees"
path1 = os.path.join(chemin_donnees, "Classement des films les plus diffusés sur les chaînes nationales gratuites depuis 1957", "Films les plus diffusés à la télévision.xlsx")
path2 = os.path.join(chemin_donnees, "Données externes", "TMDb_Dataset.csv")
df = pd.read_excel(path1, sheet_name="Classement depuis 1957", header=6)
df2 = pd.read_csv(path2)
df

,titre,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,"DELPHINE 1, YVAN 0",FARRUGIA Dominique,1996.0,FRANCE,2023.0,70.0,5.0,2.692308
1,KIRIKOU ET LES BETES SAUVAGES,OCELOT Michel / GALUP Bénédicte,2004.0,FRANCE,2021.0,69.0,24.0,4.600000
2,DOUZE TRAVAUX D'ASTERIX (LES),GOSCINNY René / UDERZO Albert,1975.0,FRANCE,2023.0,60.0,45.0,1.666667
3,ASTERIX ET LA SURPRISE DE CESAR,BRIZZI Gaëtan / BRIZZI Paul,1985.0,FRANCE,2023.0,58.0,51.0,1.611111
4,ASTERIX ET CLEOPATRE,GOSCINNY René / UDERZO Albert,1968.0,FRANCE,2023.0,58.0,44.0,1.450000
...,...,...,...,...,...,...,...,...
256,ET DIEU CREA LA FEMME,VADIM Roger,1956.0,FRANCE,2018.0,30.0,10.0,0.731707
257,JEUNES ANNEES D'UNE REINE (LES),MARISCHKA Ernst,1955.0,AUTRICHE,2023.0,30.0,10.0,0.566038
258,1 Toutes chaînes nationales gratuites confondues.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,2 La nationalité d’un film est déterminée par ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Stratégie:
On parcourt tous les films du df des meilleures audiences
Pour chaque film:
- Si on le retrouve dans les données externes, on récupère cette ligne
- Sinon on le traduit en anglais et on récupère cette ligne


In [ ]:
import requests
import wikipediaapi
import pandas as pd

def capitalise(s):
    if not s:
        return s
    s_lower = s.lower().strip()
    match = re.search(r'\s*\(([^\)]+)\)\s*$', s_lower)
    if match:
        article = match.group(1).strip()
        s_main = s_lower[:match.start()].strip()
        new_title = article.capitalize() + " " + s_main
        return new_title
    else:
        return s_lower[0].upper() + s_lower[1:]

# Fonction pour effectuer une recherche sur Wikipédia
def search_wikipedia(query, lang="fr"):
    URL = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "opensearch",
        "search": query,
        "limit": 1,         # On prend le 1er résultat
        "namespace": 0,
        "format": "json"
    }
    headers = {
        "User-Agent": "MonScript/1.0 (https://votre-site.com/contact)"
    }
    response = requests.get(URL, params=params, headers=headers)
    data = response.json()
    if len(data) >= 2 and data[1]:
        return data[1][0]
    return None

wiki_fr = wikipediaapi.Wikipedia(
    language='fr',
    user_agent='MonScript/1.0 (https://votre-site.com/contact)'
)

# Colonnes supplémentaires à ajouter dans df_final
colonnes_sup = ["réalisateur(s)", "année de production", "groupe de nationalité2",
                "année de dernière diffusion", "nb. de diffusions", "dont en première partie de soirée",
                "nb. moyen de diffusions par an"]

# df_final contiendra les colonnes de df2 plus colonnes_sup
colonnes_finales = df2.columns.tolist() + colonnes_sup

df_final = pd.DataFrame(columns=colonnes_finales)
# Pour df_not_found, on conserve uniquement les colonnes d'origine de df
df_not_found = pd.DataFrame(columns=df.columns.tolist())

found = 0
not_found = 0

for index, row in df.iterrows():
    print(f'\r{index}/{len(df)}\t\t', end='')
    titre_fr = row['titre']

    # Recherche dans df2 avec le titre tel quel
    matching_rows = df2[df2['Title'] == titre_fr]
    if not matching_rows.empty:
        row_to_add = matching_rows.iloc[[0]].copy()
        # Compléter avec les colonnes supplémentaires, si présentes dans row
        for col in colonnes_sup:
            row_to_add[col] = row[col] if col in row.index else None
        row_to_add = row_to_add.reindex(columns=colonnes_finales)
        df_final = pd.concat([df_final, row_to_add], ignore_index=True)
    else:
        # Première recherche sur Wikipédia avec le titre tel quel
        requete = f"{titre_fr}"
        titre_recherche = search_wikipedia(requete, lang="fr")

        # Si rien trouvé, on essaie après modification (capitalise)
        if not titre_recherche:
            titre_modifie = capitalise(titre_fr)
            requete = f"{titre_modifie}"
            titre_recherche = search_wikipedia(requete, lang="fr")

        if titre_recherche:
            page_fr = wiki_fr.page(titre_recherche)
            if page_fr.exists():
                if 'en' in page_fr.langlinks:
                    page_en = page_fr.langlinks['en']
                    titre_en = page_en.title
                    matching_rows = df2[df2['Title'] == titre_en]
                    if not matching_rows.empty:
                        row_to_add = matching_rows.iloc[[0]].copy()
                        for col in colonnes_sup:
                            row_to_add[col] = row[col] if col in row.index else None
                        row_to_add = row_to_add.reindex(columns=colonnes_finales)
                        df_final = pd.concat([df_final, row_to_add], ignore_index=True)
                        found += 1
                    else:
                        # Si aucune correspondance dans df2 pour le titre anglais, ajouter la ligne d'origine dans df_not_found
                        row_to_add = pd.DataFrame([row])
                        df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                        not_found += 1
                else:
                    # Pas de version anglaise disponible
                    row_to_add = pd.DataFrame([row])
                    df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                    not_found += 1
            else:
                row_to_add = pd.DataFrame([row])
                df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                not_found += 1
        else:
            row_to_add = pd.DataFrame([row])
            df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
            not_found += 1

print("\nNombre de pages trouvées :", found)
print("Nombre de pages non trouvées :", not_found)


0/261		

<ipython-input-7-e2cd999c46e3>:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)


1/261		

<ipython-input-7-e2cd999c46e3>:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


2/261		

<ipython-input-7-e2cd999c46e3>:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


260/261		
Nombre de pages trouvées : 115
Nombre de pages non trouvées : 146


In [ ]:
df_final

,TMDb_id,IMDB_id,Title,Categories,Date,Country,Runtime,Cast,Crew,First_Man_Age,...,Keywords,Poster,Bechdel,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,13679,455142,Kirikou and the Wild Beasts,"['Adventure', 'Animation', 'Family', 'Fantasy']",2005-12-07,FR,75,"[{'adult': False, 'gender': 0, 'id': 937640, '...","[{'adult': False, 'gender': 0, 'id': 59447, 'k...",NaN,...,"[{'id': 187056, 'name': 'woman director'}]",https://image.tmdb.org/t/p/w185/ixTVgA2TtUCnYL...,NaN,OCELOT Michel / GALUP Bénédicte,2004.0,FRANCE,2021.0,69.0,24.0,4.600000
1,9385,72901,The Twelve Tasks of Asterix,"['Family', 'Animation', 'Comedy', 'Adventure']",1976-06-26,FR,82,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 24530, 'k...",49.0,...,"[{'id': 588, 'name': 'rome, italy'}, {'id': 13...",https://image.tmdb.org/t/p/w185/7cZQZOYZFJcLjZ...,1.0,GOSCINNY René / UDERZO Albert,1975.0,FRANCE,2023.0,60.0,45.0,1.666667
2,9929,62687,Asterix and Cleopatra,"['Family', 'Animation', 'Adventure', 'Comedy']",1968-12-16,BE,72,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 24530, 'k...",41.0,...,"[{'id': 487, 'name': 'cairo'}, {'id': 931, 'na...",https://image.tmdb.org/t/p/w185/gNpKAmWFsCiWKK...,NaN,GOSCINNY René / UDERZO Albert,1968.0,FRANCE,2023.0,58.0,44.0,1.450000
3,9318,90667,Asterix in Britain,"['Family', 'Animation', 'Adventure', 'Comedy']",1986-12-03,FR,79,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 1965, 'kn...",59.0,...,"[{'id': 588, 'name': 'rome, italy'}, {'id': 14...",https://image.tmdb.org/t/p/w185/aE0M4I9xZRM4Pz...,0.0,VAN LAMSWEERDE Pino,1985.0,FRANCE,2023.0,56.0,49.0,1.647059
4,21348,181627,Kirikou and the Sorceress,"['Fantasy', 'Adventure', 'Animation', 'Family']",1998-12-09,FR,70,"[{'adult': False, 'gender': 0, 'id': 87421, 'k...","[{'adult': False, 'gender': 0, 'id': 59447, 'k...",NaN,...,"[{'id': 196964, 'name': 'power of goodness'}, ...",https://image.tmdb.org/t/p/w185/9RNrozaagFmpm6...,2.0,OCELOT Michel,1995.0,FRANCE / BELGIQUE,2021.0,56.0,28.0,2.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,13387,1129442,Transporter 3,"['Action', 'Adventure', 'Thriller', 'Crime']",2008-11-26,FR,104,"[{'adult': False, 'gender': 2, 'id': 976, 'kno...","[{'adult': False, 'gender': 2, 'id': 59, 'know...",41.0,...,"[{'id': 627, 'name': 'killing'}, {'id': 703, '...",https://image.tmdb.org/t/p/w185/ngpLhUccj6mlvA...,0.0,MEGATON Olivier,2008.0,FRANCE,2023.0,30.0,16.0,2.307692
111,285,449088,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']",2007-05-19,US,169,"[{'adult': False, 'gender': 2, 'id': 85, 'know...","[{'adult': False, 'gender': 2, 'id': 120, 'kno...",44.0,...,"[{'id': 911, 'name': 'exotic island'}, {'id': ...",https://image.tmdb.org/t/p/w185/8ORuWcrYPgjwUD...,3.0,VERBINSKI Gore,2007.0,ETATS UNIS,2023.0,30.0,14.0,2.000000
112,58736,53403,The Cow and I,"['War', 'Comedy', 'Adventure']",1959-12-16,FR,118,"[{'adult': False, 'gender': 2, 'id': 69958, 'k...","[{'adult': False, 'gender': 2, 'id': 29731, 'k...",56.0,...,"[{'id': 1563, 'name': 'prisoner'}, {'id': 2652...",https://image.tmdb.org/t/p/w185/2OJStomiaADWhV...,NaN,VERNEUIL Henri,1959.0,FRANCE / Italie,2023.0,30.0,13.0,0.588235
113,4180,90206,Three Men and a Cradle,['Comedy'],1985-09-18,FR,106,"[{'adult': False, 'gender': 2, 'id': 35210, 'k...","[{'adult': False, 'gender': 2, 'id': 6626, 'kn...",43.0,...,"[{'id': 187056, 'name': 'woman director'}]",https://image.tmdb.org/t/p/w185/3TfJL5jbyWFUOe...,NaN,SERREAU Coline,1985.0,FRANCE,2022.0,30.0,12.0,0.833333


In [ ]:
path = os.path.join(chemin_donnees, "LucasFeatAlain.csv")
df_final.to_csv(path, index=False)

In [ ]:
df_not_found

,titre,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,"DELPHINE 1, YVAN 0",FARRUGIA Dominique,1996.0,FRANCE,2023.0,70.0,5.0,2.692308
1,ASTERIX ET LA SURPRISE DE CESAR,BRIZZI Gaëtan / BRIZZI Paul,1985.0,FRANCE,2023.0,58.0,51.0,1.611111
2,DEUX HEURES MOINS LE QUART AVANT JESUS-CHRIST,YANNE Jean,1982.0,FRANCE,2022.0,57.0,33.0,1.540541
3,SCOUT TOUJOURS,JUGNOT Gérard,1985.0,FRANCE,2023.0,56.0,37.0,1.513514
4,TINTIN ET LE LAC AUX REQUINS,LEBLANC Raymond,1972.0,BELGIQUE / FRANCE,2023.0,55.0,22.0,2.037037
...,...,...,...,...,...,...,...,...
141,ET DIEU CREA LA FEMME,VADIM Roger,1956.0,FRANCE,2018.0,30.0,10.0,0.731707
142,JEUNES ANNEES D'UNE REINE (LES),MARISCHKA Ernst,1955.0,AUTRICHE,2023.0,30.0,10.0,0.566038
143,1 Toutes chaînes nationales gratuites confondues.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,2 La nationalité d’un film est déterminée par ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def capitalise(s):
    if not s:
        return s
    # Convertir la chaîne en minuscules pour uniformiser le traitement
    s_lower = s.lower().strip()
    # Chercher un motif de la forme " (quelquechose)" en fin de chaîne
    match = re.search(r'\s*\(([^\)]+)\)\s*$', s_lower)
    if match:
        # Récupérer le contenu entre parenthèses (par exemple "les", "la", "le", etc.)
        article = match.group(1).strip()
        # Supprimer la partie entre parenthèses de la chaîne
        s_main = s_lower[:match.start()].strip()
        # Construire le nouveau titre : l'article (avec la première lettre en majuscule) suivi du reste
        new_title = article.capitalize() + " " + s_main
        return new_title
    else:
        # Sinon, simplement mettre la première lettre en majuscule et le reste en minuscules
        return s_lower[0].upper() + s_lower[1:]

# Exemple avec la première valeur de la colonne "titre" de df_not_found
titre1 = df_not_found["titre"].iloc[6]  # Utilisation de .iloc pour un accès correct
titre1 = capitalise(titre1)
print("Titre après capitalisation :", titre1)

requete = f"{titre1}"
titre_recherche = search_wikipedia(requete, lang="fr")

if titre_recherche:
      page_fr = wiki_fr.page(titre_recherche)
      if page_fr.exists():
        print("OK francais")
        if 'en' in page_fr.langlinks:
          print("OK anglais")
          page_en = page_fr.langlinks['en']
          titre_en = page_en.title
          print(titre_en)

Titre après capitalisation : Asterix et le coup du menhir
OK francais
OK anglais
Asterix and the Big Fight (film)


In [ ]:
df_not_found

,titre,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,"DELPHINE 1, YVAN 0",FARRUGIA Dominique,1996.0,FRANCE,2023.0,70.0,5.0,2.692308
1,ASTERIX ET LA SURPRISE DE CESAR,BRIZZI Gaëtan / BRIZZI Paul,1985.0,FRANCE,2023.0,58.0,51.0,1.611111
2,DEUX HEURES MOINS LE QUART AVANT JESUS-CHRIST,YANNE Jean,1982.0,FRANCE,2022.0,57.0,33.0,1.540541
3,SCOUT TOUJOURS,JUGNOT Gérard,1985.0,FRANCE,2023.0,56.0,37.0,1.513514
4,TINTIN ET LE LAC AUX REQUINS,LEBLANC Raymond,1972.0,BELGIQUE / FRANCE,2023.0,55.0,22.0,2.037037
...,...,...,...,...,...,...,...,...
141,ET DIEU CREA LA FEMME,VADIM Roger,1956.0,FRANCE,2018.0,30.0,10.0,0.731707
142,JEUNES ANNEES D'UNE REINE (LES),MARISCHKA Ernst,1955.0,AUTRICHE,2023.0,30.0,10.0,0.566038
143,1 Toutes chaînes nationales gratuites confondues.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,2 La nationalité d’un film est déterminée par ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for title in df2["Title"]:
  if "asterix" in title.lower():
    print(title)

Asterix & Obelix: Mission Cleopatra
Asterix vs. Caesar
Asterix in Britain
Asterix Conquers America
The Twelve Tasks of Asterix
Asterix & Obelix Take on Caesar
Asterix and the Vikings
Asterix and Cleopatra
Asterix the Gaul
Asterix and the Big Fight
Asterix & Obelix: God Save Britannia
Asterix: The Mansions of the Gods
Asterix: The Secret of the Magic Potion


In [ ]:
import requests
import wikipediaapi
import pandas as pd

def similar_titles(title1, title2, threshold=80):
    return fuzz.token_set_ratio(title1, title2) > threshold

def capitalise(s):
    if not s:
        return s
    s_lower = s.lower().strip()
    match = re.search(r'\s*\(([^\)]+)\)\s*$', s_lower)
    if match:
        article = match.group(1).strip()
        s_main = s_lower[:match.start()].strip()
        new_title = article.capitalize() + " " + s_main
        return new_title
    else:
        return s_lower[0].upper() + s_lower[1:]

# Fonction pour effectuer une recherche sur Wikipédia
def search_wikipedia(query, lang="fr"):
    URL = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "opensearch",
        "search": query,
        "limit": 1,         # On prend le 1er résultat
        "namespace": 0,
        "format": "json"
    }
    headers = {
        "User-Agent": "MonScript/1.0 (https://votre-site.com/contact)"
    }
    response = requests.get(URL, params=params, headers=headers)
    data = response.json()
    if len(data) >= 2 and data[1]:
        return data[1][0]
    return None

wiki_fr = wikipediaapi.Wikipedia(
    language='fr',
    user_agent='MonScript/1.0 (https://votre-site.com/contact)'
)

# Colonnes supplémentaires à ajouter dans df_final
colonnes_sup = ["réalisateur(s)", "année de production", "groupe de nationalité2",
                "année de dernière diffusion", "nb. de diffusions", "dont en première partie de soirée",
                "nb. moyen de diffusions par an"]

# df_final contiendra les colonnes de df2 plus colonnes_sup
colonnes_finales = df2.columns.tolist() + colonnes_sup

df_final = pd.DataFrame(columns=colonnes_finales)
# Pour df_not_found, on conserve uniquement les colonnes d'origine de df
df_not_found = pd.DataFrame(columns=df.columns.tolist())

found = 0
not_found = 0

for index, row in df.iterrows():
    print(f'\r{index}/{len(df)}\t\t', end='')
    titre_fr = row['titre']

    # Recherche dans df2 avec le titre tel quel
    matching_rows = df2[df2['Title'].apply(lambda x: similar_titles(x, titre_fr))]
    if not matching_rows.empty:
        row_to_add = matching_rows.iloc[[0]].copy()
        # Compléter avec les colonnes supplémentaires, si présentes dans row
        for col in colonnes_sup:
            row_to_add[col] = row[col] if col in row.index else None
        row_to_add = row_to_add.reindex(columns=colonnes_finales)
        df_final = pd.concat([df_final, row_to_add], ignore_index=True)
    else:
        # Première recherche sur Wikipédia avec le titre tel quel
        requete = f"{titre_fr}"
        titre_recherche = search_wikipedia(requete, lang="fr")

        # Si rien trouvé, on essaie après modification (capitalise)
        if not titre_recherche:
            titre_modifie = capitalise(titre_fr)
            requete = f"{titre_modifie}"
            titre_recherche = search_wikipedia(requete, lang="fr")

        if titre_recherche:
            page_fr = wiki_fr.page(titre_recherche)
            if page_fr.exists():
                if 'en' in page_fr.langlinks:
                    page_en = page_fr.langlinks['en']
                    titre_en = page_en.title
                    matching_rows = df2[df2['Title'].apply(lambda x: similar_titles(x, titre_en))]
                    if not matching_rows.empty:
                        row_to_add = matching_rows.iloc[[0]].copy()
                        for col in colonnes_sup:
                            row_to_add[col] = row[col] if col in row.index else None
                        row_to_add = row_to_add.reindex(columns=colonnes_finales)
                        df_final = pd.concat([df_final, row_to_add], ignore_index=True)
                        found += 1
                    else:
                        # Si aucune correspondance dans df2 pour le titre anglais, ajouter la ligne d'origine dans df_not_found
                        row_to_add = pd.DataFrame([row])
                        df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                        not_found += 1
                else:
                    # Pas de version anglaise disponible
                    row_to_add = pd.DataFrame([row])
                    df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                    not_found += 1
            else:
                row_to_add = pd.DataFrame([row])
                df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
                not_found += 1
        else:
            row_to_add = pd.DataFrame([row])
            df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)
            not_found += 1

print("\nNombre de pages trouvées :", found)
print("Nombre de pages non trouvées :", not_found)


0/261		

<ipython-input-14-4c0faa7383e2>:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_not_found = pd.concat([df_not_found, row_to_add], ignore_index=True)


1/261		

<ipython-input-14-4c0faa7383e2>:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


2/261		

<ipython-input-14-4c0faa7383e2>:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, row_to_add], ignore_index=True)


260/261		
Nombre de pages trouvées : 184
Nombre de pages non trouvées : 72


In [ ]:
df_final

,TMDb_id,IMDB_id,Title,Categories,Date,Country,Runtime,Cast,Crew,First_Man_Age,...,Keywords,Poster,Bechdel,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,13679,455142,Kirikou and the Wild Beasts,"['Adventure', 'Animation', 'Family', 'Fantasy']",2005-12-07,FR,75,"[{'adult': False, 'gender': 0, 'id': 937640, '...","[{'adult': False, 'gender': 0, 'id': 59447, 'k...",NaN,...,"[{'id': 187056, 'name': 'woman director'}]",https://image.tmdb.org/t/p/w185/ixTVgA2TtUCnYL...,NaN,OCELOT Michel / GALUP Bénédicte,2004.0,FRANCE,2021.0,69.0,24.0,4.600000
1,9385,72901,The Twelve Tasks of Asterix,"['Family', 'Animation', 'Comedy', 'Adventure']",1976-06-26,FR,82,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 24530, 'k...",49.0,...,"[{'id': 588, 'name': 'rome, italy'}, {'id': 13...",https://image.tmdb.org/t/p/w185/7cZQZOYZFJcLjZ...,1.0,GOSCINNY René / UDERZO Albert,1975.0,FRANCE,2023.0,60.0,45.0,1.666667
2,8868,88748,Asterix vs. Caesar,"['Comedy', 'Animation', 'Family', 'Adventure']",1985-03-06,FR,79,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 1965, 'kn...",58.0,...,"[{'id': 588, 'name': 'rome, italy'}, {'id': 13...",https://image.tmdb.org/t/p/w185/cfS5T0GGvYcD18...,0.0,BRIZZI Gaëtan / BRIZZI Paul,1985.0,FRANCE,2023.0,58.0,51.0,1.611111
3,2899,250223,Asterix & Obelix: Mission Cleopatra,"['Family', 'Fantasy', 'Comedy', 'Adventure']",2002-01-30,FR,108,"[{'adult': False, 'gender': 2, 'id': 28781, 'k...","[{'adult': False, 'gender': 2, 'id': 4275, 'kn...",50.0,...,"[{'id': 1160, 'name': 'egypt'}, {'id': 2343, '...",https://image.tmdb.org/t/p/w185/fKIhO8pL7pVcIG...,3.0,GOSCINNY René / UDERZO Albert,1968.0,FRANCE,2023.0,58.0,44.0,1.450000
4,9318,90667,Asterix in Britain,"['Family', 'Animation', 'Adventure', 'Comedy']",1986-12-03,FR,79,"[{'adult': False, 'gender': 2, 'id': 34676, 'k...","[{'adult': False, 'gender': 2, 'id': 1965, 'kn...",59.0,...,"[{'id': 588, 'name': 'rome, italy'}, {'id': 14...",https://image.tmdb.org/t/p/w185/aE0M4I9xZRM4Pz...,0.0,VAN LAMSWEERDE Pino,1985.0,FRANCE,2023.0,56.0,49.0,1.647059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,16520,49408,The King and I,"['Drama', 'Romance', 'Adventure', 'Music']",1956-06-29,US,133,"[{'adult': False, 'gender': 1, 'id': 20141, 'k...","[{'adult': False, 'gender': 2, 'id': 2637, 'kn...",36.0,...,"[{'id': 1530, 'name': 'temple'}, {'id': 2569, ...",https://image.tmdb.org/t/p/w185/wUfaP0lLaMpZzp...,NaN,VERNEUIL Henri,1959.0,FRANCE / Italie,2023.0,30.0,13.0,0.588235
185,4180,90206,Three Men and a Cradle,['Comedy'],1985-09-18,FR,106,"[{'adult': False, 'gender': 2, 'id': 35210, 'k...","[{'adult': False, 'gender': 2, 'id': 6626, 'kn...",43.0,...,"[{'id': 187056, 'name': 'woman director'}]",https://image.tmdb.org/t/p/w185/3TfJL5jbyWFUOe...,NaN,SERREAU Coline,1985.0,FRANCE,2022.0,30.0,12.0,0.833333
186,8489,248667,Ali,['Drama'],2001-12-10,US,157,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 2, 'id': 27, 'know...",33.0,...,"[{'id': 396, 'name': 'transporter'}, {'id': 27...",https://image.tmdb.org/t/p/w185/aBphVTRYpJS028...,NaN,BECKER Jacques,1954.0,FRANCE,2022.0,30.0,11.0,0.468750
187,8420,49189,...And God Created Woman,"['Drama', 'Romance']",1956-11-28,Inconnu,95,"[{'adult': False, 'gender': 1, 'id': 3783, 'kn...","[{'adult': False, 'gender': 2, 'id': 2579, 'kn...",26.0,...,"[{'id': 2087, 'name': 'lolita'}, {'id': 2393, ...",https://image.tmdb.org/t/p/w185/viZwV6ji5txsyr...,NaN,VADIM Roger,1956.0,FRANCE,2018.0,30.0,10.0,0.731707


In [ ]:
df_not_found

,titre,réalisateur(s),année de production,groupe de nationalité2,année de dernière diffusion,nb. de diffusions,dont en première partie de soirée,nb. moyen de diffusions par an
0,"DELPHINE 1, YVAN 0",FARRUGIA Dominique,1996.0,FRANCE,2023.0,70.0,5.0,2.692308
1,DEUX HEURES MOINS LE QUART AVANT JESUS-CHRIST,YANNE Jean,1982.0,FRANCE,2022.0,57.0,33.0,1.540541
2,TINTIN ET LE TEMPLE DU SOLEIL,LEBLANC Raymond,1969.0,BELGIQUE / FRANCE,2023.0,55.0,19.0,2.037037
3,SOUS-DOUES (LES),ZIDI Claude,1979.0,FRANCE,2023.0,51.0,33.0,1.275000
4,BOUGE !,CORNUAU Jérôme,1997.0,FRANCE,2023.0,51.0,10.0,2.040000
...,...,...,...,...,...,...,...,...
67,RANDONNEURS (LES),HAREL Philippe,1996.0,FRANCE,2023.0,30.0,20.0,1.200000
68,INCONTROLABLE,SHART Raffy,2005.0,FRANCE,2019.0,30.0,18.0,2.500000
69,1 Toutes chaînes nationales gratuites confondues.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2 La nationalité d’un film est déterminée par ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(capitalise("SOUS-DOUES (LES)	"))

Les sous-doues
